# CAH 30503 — Week 3: Technical Prototype Strategy

**Theme**: From "there are thousands of AI models" to "I know which ones I need and what my app will and won't do."

---

This is the first real code week. You'll run AI pipelines, compare models, read model cards, and scope your MVP. By the end, you'll have a technical plan for what to build next week.

And you'll learn the name for the loop you've been practicing for two weeks.

## Setup

Run this cell first. It installs everything we need and may take a minute.

In [ ]:
!pip install -q transformers torch huggingface_hub

import torch
from transformers import pipeline
from huggingface_hub import list_models

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
print("Setup complete!")

---

## Activity 1: The Model Ecosystem

There are 500,000+ free AI models on the [Hugging Face Hub](https://huggingface.co/models). You don't need to browse all of them — you need to find the ones relevant to your project.

### Browse Models by Task

Run the cell below to see the most popular models for different tasks.

In [ ]:
# See popular models for common task types
tasks = ["text-classification", "summarization", "question-answering",
         "ner", "translation", "text-generation"]

for task in tasks:
    models = list(list_models(task=task, sort="downloads", limit=3))
    print(f"\n{'='*50}")
    print(f"Task: {task}")
    print(f"{'='*50}")
    for m in models:
        downloads = f"{m.downloads:,}" if m.downloads else "N/A"
        print(f"  {m.id}  ({downloads} downloads)")

**Which task type(s) are relevant to your project?** 

*(Think about your problem from Week 2: what does the AI need to DO? Summarize? Classify? Answer questions? Extract names?)*



### Your First Pipeline: Sentiment Analysis

A **pipeline** is like a function that does one AI task. One line of code. Run it.

In [ ]:
# Create a sentiment analysis pipeline
classifier = pipeline("sentiment-analysis", device=device)

# Three test sentences — watch how it handles each one
sentences = [
    "I absolutely love this product, it changed my life!",    # clearly positive
    "This is the worst experience I've ever had.",             # clearly negative
    "Well, I guess it could have been worse.",                 # ambiguous
]

for s in sentences:
    result = classifier(s)
    label = result[0]["label"]
    score = round(result[0]["score"], 3)
    print(f"  '{s}'")
    print(f"  → {label} (confidence: {score})")
    print()

**Look at the third result** — the ambiguous sentence. What label did the model give it? What confidence? Do you agree with the model?

*(Write your observation here)*



### Second Pipeline: Summarization

Before running the model, **summarize this paragraph yourself in one sentence**:

> The invention of the printing press in the 15th century fundamentally transformed European society. Before Gutenberg, books were copied by hand, making them expensive and rare. The printing press made books affordable, which dramatically increased literacy rates across social classes. This democratization of knowledge contributed to the Protestant Reformation, the Scientific Revolution, and eventually the Enlightenment. Scholars could share ideas across borders, building on each other's work in ways that were previously impossible.

**Your one-sentence summary**: 



In [ ]:
# Now let the model summarize the same paragraph
summarizer = pipeline("summarization", device=device)

text = (
    "The invention of the printing press in the 15th century fundamentally "
    "transformed European society. Before Gutenberg, books were copied by hand, "
    "making them expensive and rare. The printing press made books affordable, "
    "which dramatically increased literacy rates across social classes. This "
    "democratization of knowledge contributed to the Protestant Reformation, "
    "the Scientific Revolution, and eventually the Enlightenment. Scholars could "
    "share ideas across borders, building on each other's work in ways that were "
    "previously impossible."
)

result = summarizer(text, max_length=60, min_length=15)
print("Model's summary:")
print(result[0]["summary_text"])

**Compare your summary to the model's**: What did the model keep? What did it lose? What would you have kept that it dropped?

*(Write your comparison here)*



### Free Exploration: Try a Pipeline for Your Project

Pick a pipeline type relevant to YOUR project and try it on your own input.

In [ ]:
# Try a pipeline relevant to your project
# Change the task name and input to match your needs
#
# Available tasks:
#   "sentiment-analysis"  — positive/negative classification
#   "summarization"       — condense long text
#   "ner"                 — extract names, places, organizations
#   "question-answering"  — answer questions about a passage
#   "text-generation"     — complete/generate text
#   "translation_en_to_fr" — translate English to French

# Example: Named Entity Recognition
my_pipe = pipeline("ner", aggregation_strategy="simple", device=device)
my_input = "Barack Obama visited the Eiffel Tower in Paris last summer."

results = my_pipe(my_input)
print(f"Input: '{my_input}'")
print("\nEntities found:")
for r in results:
    print(f"  {r['word']} → {r['entity_group']} (confidence: {round(r['score'], 3)})")

In [ ]:
# Now try YOUR input — change the task and text below

# my_pipe = pipeline("YOUR-TASK-HERE", device=device)
# my_input = "YOUR INPUT TEXT HERE"
# result = my_pipe(my_input)
# print(result)

---

## Activity 2: Model Comparison with Plan Template

Now we get structured. Instead of just "trying stuff," you'll **plan a comparison**, execute it, and analyze the results.

### Your Comparison Plan

Fill this in BEFORE running anything:

- **Task**: *(What pipeline type am I investigating?)*

- **Question**: *(What am I trying to find out?)*

- **Models to compare**: *(List 2-3 models)*

- **Inputs**: *(What test inputs will I use? Why these?)*

- **Criteria**: *(What am I measuring? Speed? Quality? Relevance to my users?)*

- **Expected outcome**: *(What do I THINK I'll find? Write this BEFORE running.)*


### Model Card Literacy

Before comparing models, read the **model cards** for the models you're considering.

Go to [huggingface.co/models](https://huggingface.co/models), search for your model, and answer the 5 key questions:

A model card is like a nutrition label for AI — it tells you what's inside.

#### Model 1: _____________

1. **What was it trained on?** *(dataset, domain, language)*

2. **What is it intended for?**

3. **What are the known limitations?**

4. **How was it evaluated?**

5. **Who made it and when?**

#### Model 2: _____________

1. **Trained on?**

2. **Intended for?**

3. **Limitations?**

4. **Evaluated how?**

5. **Who/when?**


### Head-to-Head Comparison

Run your models on the same inputs and compare. Modify the code below for your task.

In [ ]:
# Head-to-head model comparison
# Example: comparing two sentiment models
# Change the models and inputs to match YOUR plan template

model_a = pipeline("sentiment-analysis",
                   model="distilbert-base-uncased-finetuned-sst-2-english",
                   device=device)

model_b = pipeline("sentiment-analysis",
                   model="nlptown/bert-base-multilingual-uncased-sentiment",
                   device=device)

test_inputs = [
    "This restaurant has amazing food and great service!",
    "The food was okay but nothing special.",
    "I waited an hour and the order was wrong.",
]

print("Model A: distilbert (binary: POSITIVE/NEGATIVE)")
print("Model B: nlptown (5-star rating)")
print("=" * 60)

for text in test_inputs:
    result_a = model_a(text)[0]
    result_b = model_b(text)[0]
    print(f"\nInput: '{text}'")
    print(f"  Model A: {result_a['label']} ({round(result_a['score'], 3)})")
    print(f"  Model B: {result_b['label']} ({round(result_b['score'], 3)})")

In [ ]:
# YOUR comparison — modify for your project's task and models

# model_a = pipeline("YOUR-TASK", model="MODEL-A-NAME", device=device)
# model_b = pipeline("YOUR-TASK", model="MODEL-B-NAME", device=device)
#
# test_inputs = [
#     "Your first test input",
#     "Your second test input",
#     "Your third test input",
# ]
#
# for text in test_inputs:
#     result_a = model_a(text)
#     result_b = model_b(text)
#     print(f"Input: '{text}'")
#     print(f"  Model A: {result_a}")
#     print(f"  Model B: {result_b}")
#     print()

### What I Found

- **Result**: *(Which model "won"? On what criteria?)*

- **Surprise**: *(Anything unexpected?)*

- **My choice for my project**: *(Model X because...)*

- **What the model card told me that the output didn't**: *(from your model card reading)*



---

## The PDER Reveal

Look at what you just did:

1. You had a question → **Plan**
2. You ran the models → **Direct**
3. You compared results to expectations → **Examine**
4. You're about to write down what you learned → **Record**

**You've been doing this loop since Week 1.**

```
    PLAN → DIRECT → EXAMINE → RECORD
     ↑                          │
     └──────────────────────────┘
```

It's called **PDER** — Plan, Direct, Examine, Record.

- Week 1: You tried AI tools and wrote observations (implicit loop)
- Week 2: You planned research questions and compared assumptions to reality (deliberate loop)
- Week 3: You wrote a plan template, ran a comparison, and analyzed results (structured loop)

The loop doesn't change. What changes is how **sophisticated** each step gets.

**Record is the step that makes the whole loop compound.** Every time you write down what you learned, the next cycle starts from a higher baseline. Your CLAUDE.md file IS the Record step, accumulated over 8 weeks.

---

## Activity 3: Scope the Build

### Feature Brainstorm + Impact/Effort Matrix

List every feature your app could have. Then map them:

```
         High Impact
              │
    ┌─────────┼─────────┐
    │  DO      │  PLAN    │
    │  FIRST   │  CAREFULLY│
    │  (quick  │  (worth   │
    │   wins)  │   the     │
    │          │   effort) │
Low ──────────┼──────────── High
Effort        │           Effort
    │  MAYBE   │  SKIP     │
    │  LATER   │  (tempting│
    │          │   but not  │
    │          │   worth it)│
    └─────────┼─────────┘
         Low Impact
```

**My features mapped to quadrants:**

| Feature | Impact | Effort | Quadrant |
|---------|--------|--------|----------|
| | | | |
| | | | |
| | | | |
| | | | |
| | | | |


### MVP Definition

The **Minimum Viable Product** is the smallest version someone would actually use more than once.

**The Adoption Threshold** (from Week 2 research):
For my target users, the threshold is:


**MVP Features** (must have — 3 maximum):
1. 
2. 
3. 

**Version 2** (after MVP works):
1. 
2. 

**Explicitly NOT Building** (tempting but cut):
1. 
2. 


---

## DCS Question: What Knowledge Is Encoded in the Models I Chose?

Based on the model cards you read:

- **What training data shaped my model?**

- **Whose decisions are embedded in it?**

- **What domains or perspectives might it underrepresent?**

*(Write 2-3 sentences grounded in the specific model card you read, not generic statements about AI)*



---

## Record: CLAUDE.md Week 3 Entry

Add this to your CLAUDE.md file:

```
## Week 3: Technical Prototype Strategy

### PDER Named
The loop I've been practicing: Plan → Direct → Examine → Record.
Plan = decide what to investigate. Direct = run it.
Examine = compare results to expectations. Record = write it down.

### Pipeline Pattern
from transformers import pipeline
pipe = pipeline("task-name")
result = pipe("my input")
Pipelines I explored: [list]
Most relevant to my project: [which and why]

### Model Selection
For [task], I compared [models]. My choice: [X] because [Y].
Model card: trained on [data], intended for [use], limited by [limitation].

### MVP Spec
MVP features: [3 must-haves]
Explicitly not building: [what was cut and why]
Adoption threshold: [from Week 2, refined]

### DCS: What Knowledge Is Encoded?
[What training data, whose decisions, what's underrepresented]
```

---

## What's Next

You have a plan, a model, and a scope. You know what you're building, what you're skipping, and what AI tools you'll use.

**Next week**: You build it. You'll direct Claude Code to create your application, wrap it in a Gradio interface (a real app with a UI), and watch someone else use it. You'll also learn the 6-question examination protocol — a structured way to evaluate what you built.